## Лабораторная работа 3

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
from pyspark.sql.functions import col, pandas_udf, split, lower, udf
from pyspark.sql.types import LongType, StructType, StructField, IntegerType, StringType, DoubleType

conf = SparkConf()
conf.set("spark.app.name", "Spark Lab 3") 

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2021-02-27 22:12 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2021-02-27 22:12 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2021-02-27 22:12 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2021-02-27 22:12 /labs/slaba03/laba03_views_programmes.csv


In [5]:
items = spark.read.csv('/labs/slaba03/laba03_items.csv', header=True, sep='\t')
items.registerTempTable("items")

In [6]:
items.limit(2).toPandas()

,item_id,channel_id,datetime_availability_start,datetime_availability_stop,datetime_show_start,datetime_show_stop,content_type,title,year,genres,region_id
0,7006336,9.0,1970-01-01T00:00:00Z,2017-04-04T09:00:00Z,2017-04-01T08:05:00Z,2017-04-01T09:00:00Z,0,крылатый властелин морей,None,General,41.0
1,7006349,9.0,1970-01-01T00:00:00Z,2017-04-04T20:55:00Z,2017-04-01T20:00:00Z,2017-04-01T20:55:00Z,0,крылатый властелин морей,None,General,41.0


In [7]:
test = spark.read.csv('/labs/slaba03/laba03_test.csv', header=True)
test.registerTempTable("test")

In [8]:
test.limit(2).toPandas()

,user_id,item_id,purchase
0,1654,94814,None
1,1654,93629,None


In [9]:
train = spark.read.csv('/labs/slaba03/laba03_train.csv', header=True)
train.registerTempTable("train")

In [10]:
train.limit(2).toPandas()

,user_id,item_id,purchase
0,903319,11005,0
1,903319,88884,0


In [11]:
views = spark.read.csv('/labs/slaba03/laba03_views_programmes.csv', header=True)
views.registerTempTable("views")

In [12]:
views.limit(2).toPandas()

,user_id,item_id,ts_start,ts_end,item_type
0,0,7101053,1491409931,1491411600,live
1,0,7101054,1491412481,1491451571,live


In [13]:
# соотношение 0 и 1
train.groupBy("purchase").count().collect()

[Row(purchase='0', count=5021720), Row(purchase='1', count=10904)]

In [14]:
10904 / (5021720 + 10904)

0.0021666629575346776

### Feature Engineering

Создаем обучающую выборку

Features: 

* year (NA = 1970)
* genre_id (NA = 'NA')
* mean_user_purchase
* mean_id_purchase

In [90]:
train_temp = train.join(items, on="item_id", how="left")
train_temp.registerTempTable("train_temp")

In [91]:
train_temp = train_temp.select('user_id',
                 'item_id', 'purchase',
                 'year','genres')

In [92]:
train_temp.show(5)

+-------+-------+--------+------+-------+
|user_id|item_id|purchase|  year| genres|
+-------+-------+--------+------+-------+
| 865812| 100140|       0|2014.0|Комедии|
| 866037| 100140|       0|2014.0|Комедии|
| 866155| 100140|       0|2014.0|Комедии|
| 866207| 100140|       0|2014.0|Комедии|
| 866320| 100140|       0|2014.0|Комедии|
+-------+-------+--------+------+-------+
only showing top 5 rows



In [93]:
# обрабатываем категориальную переменную genres
from pyspark.ml.feature import StringIndexer

train_temp = train_temp.na.fill({'genres':'NA'}) # заменяем пропуски
indexer = StringIndexer(inputCol="genres", outputCol="genre_id") 
train_temp = indexer.fit(train_temp).transform(train_temp) 
train_temp = train_temp.drop("genres").cache()
train_temp.show(2)

+-------+-------+--------+------+--------+
|user_id|item_id|purchase|  year|genre_id|
+-------+-------+--------+------+--------+
|   1654| 100140|       0|2014.0|    12.0|
| 510087| 100140|       0|2014.0|    12.0|
+-------+-------+--------+------+--------+
only showing top 2 rows



In [94]:
# проверяем, что нет пропусков
train_temp.where(col("genre_id").isNull()).show()

+-------+-------+--------+----+--------+
|user_id|item_id|purchase|year|genre_id|
+-------+-------+--------+----+--------+
+-------+-------+--------+----+--------+



In [95]:
item_popularity = spark.sql("""
    SELECT 
        item_id,
        sum(purchase) / count(item_id) AS mean_item_purchase
    FROM train_temp
    GROUP BY item_id
""")

item_popularity.show(5)

+-------+--------------------+
|item_id|  mean_item_purchase|
+-------+--------------------+
|  98725|7.369196757553427E-4|
|    691|                 0.0|
|   2136| 7.22543352601156E-4|
|  88649|0.001454545454545...|
|  74605|                 0.0|
+-------+--------------------+
only showing top 5 rows



In [96]:
user_popularity = spark.sql("""
    SELECT 
        user_id,
        sum(purchase) / count(user_id) AS mean_user_purchase
    FROM train_temp
    GROUP BY user_id
""")

user_popularity.show(5)

+-------+--------------------+
|user_id|  mean_user_purchase|
+-------+--------------------+
| 921852|3.789314134141720...|
| 927169|0.003464203233256351|
| 929499|0.002714230321830167|
| 930508|0.002693343593689...|
| 867363|3.892565200467107...|
+-------+--------------------+
only showing top 5 rows



In [97]:
train_temp = train_temp.join(item_popularity, on="item_id", how="left")
train_temp = train_temp.join(user_popularity, on="user_id", how="left")

In [99]:
train_temp.show(2)

+-------+------------------+-------+------------------+-----+----+--------+
|item_id|mean_item_purchase|user_id|mean_user_purchase|label|year|genre_id|
+-------+------------------+-------+------------------+-----+----+--------+
|  11236|      0.0014869889| 867363|      3.8925651E-4|    0|2010|     104|
|  66423|       7.199424E-4| 867363|      3.8925651E-4|    0|1957|     121|
+-------+------------------+-------+------------------+-----+----+--------+
only showing top 2 rows



In [101]:
train_temp = train_temp.select(F.col("item_id").cast("int"),
                               F.col("mean_item_purchase").cast("float"),
                               F.col("user_id").cast("int"),
                               F.col("mean_user_purchase").cast("float"),
                               F.col("label").cast("int").alias('label'),
                               F.col("year").cast("int"),
                               F.col("genre_id").cast("int"))                              

In [102]:
train_temp.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- mean_item_purchase: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- mean_user_purchase: float (nullable = true)
 |-- label: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- genre_id: integer (nullable = true)



**Обрабатываем пропуски**

In [103]:
train_temp = train_temp.na.fill({'item_id':-1, 'user_id':-1, 
                                 'year':1970,'label':0,
                                 'mean_user_purchase':0, 'mean_item_purchase':0})
train_temp = train_temp.cache()

**Векторизуем признаки**

In [104]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs = ['mean_item_purchase', 'mean_user_purchase', 'year', 'genre_id',]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [105]:
train_temp = assembler.transform(train_temp)

**Делим на train и test**

Сохраняем 20% нулей и единичек в отложенной выборке

In [106]:
train_1 = train_temp.sampleBy("label", fractions={0: 0.8, 1: 0.8}, seed=420) 
train_1.groupby("label").count().collect()

[Row(label=1, count=8727), Row(label=0, count=4017891)]

In [107]:
8727 / (8727 + 4017891)

0.0021673275190246507

In [108]:
test_1 = train_temp.join(train_1, on=["user_id","item_id"], how="leftanti") # удаляем train из train_temp
test_1.groupby("label").count().collect()

[Row(label=1, count=2177), Row(label=0, count=1003829)]

In [110]:
2177 / (2177 + 1003829)

0.0021640029979940476

In [109]:
test_1 = test_1.select("user_id","item_id","label", "features").cache()
train_1 = train_1.select("user_id","item_id", "label", "features").cache()

In [111]:
test_1.show(2)
train_1.show(2)

+-------+-------+-----+--------------------+
|user_id|item_id|label|            features|
+-------+-------+-----+--------------------+
|   1654|   7679|    0|[7.50750768929719...|
|   1654|  10976|    0|[0.0,0.0019470405...|
+-------+-------+-----+--------------------+
only showing top 2 rows

+-------+-------+-----+--------------------+
|user_id|item_id|label|            features|
+-------+-------+-----+--------------------+
| 867363|   1870|    0|[7.14285706635564...|
| 867363|   2696|    0|[7.34753848519176...|
+-------+-------+-----+--------------------+
only showing top 2 rows



**Обучаем логистическую регрессию**

In [112]:
from pyspark.ml.classification import LogisticRegression

In [113]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=15, regParam=0.1)
lrModel = lr.fit(train_1)

In [114]:
lrModel

LogisticRegressionModel: uid = LogisticRegression_6e196eee0f9f, numClasses = 2, numFeatures = 4

In [115]:
predictions = lrModel.transform(test_1)

In [116]:
predictions.show(2)

+-------+-------+-----+--------------------+--------------------+--------------------+----------+
|user_id|item_id|label|            features|       rawPrediction|         probability|prediction|
+-------+-------+-----+--------------------+--------------------+--------------------+----------+
|   1654|   7679|    0|[7.50750768929719...|[6.14966692418366...|[0.99787035257797...|       0.0|
|   1654|  10976|    0|[0.0,0.0019470405...|[6.15690461731163...|[0.99788567819493...|       0.0|
+-------+-------+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [117]:
predictions.select("user_id", "item_id", "label", "prediction", "probability", "rawPrediction").show(5, False, True)

-RECORD 0---------------------------------------------------
 user_id       | 1654                                       
 item_id       | 7679                                       
 label         | 0                                          
 prediction    | 0.0                                        
 probability   | [0.9978703525779781,0.0021296474220219284] 
 rawPrediction | [6.14966692418366,-6.14966692418366]       
-RECORD 1---------------------------------------------------
 user_id       | 1654                                       
 item_id       | 10976                                      
 label         | 0                                          
 prediction    | 0.0                                        
 probability   | [0.9978856781949366,0.0021143218050634293] 
 rawPrediction | [6.156904617311632,-6.156904617311632]     
-RECORD 2---------------------------------------------------
 user_id       | 1654                                       
 item_id       | 73179  

In [119]:
correct_predictions = predictions.select("label", F.col("prediction").cast("int")).filter("label == prediction").count()
correct_predictions

1003829

In [120]:
all_predictions = predictions.count()
all_predictions

1006006

In [121]:
print("Accuracy is {}".format(correct_predictions / all_predictions))

Accuracy is 0.997835997002006


In [125]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName='areaUnderROC')
f"AUC is {evaluator.evaluate(predictions)}"

'AUC is 0.927210409092778'

Собираем предсказания

In [126]:
split1_udf = udf(lambda value: value[0].item(), DoubleType())
split2_udf = udf(lambda value: value[1].item(), DoubleType())
output2 = predictions.select(split1_udf('probability').alias('c1'), split2_udf('probability').alias('c2'))

In [127]:
output2.show()

+------------------+--------------------+
|                c1|                  c2|
+------------------+--------------------+
|0.9978703525779781|0.002129647422021...|
|0.9978856781949366|0.002114321805063...|
|0.9978357509409913|0.002164249059008...|
|0.9978713531999328|0.002128646800067...|
|0.9978704640532923|0.002129535946707...|
|0.9978133028391204|0.002186697160879...|
|0.9978750668593972|0.002124933140602...|
|0.9978765004872276|0.002123499512772252|
|0.9978753510085504|0.002124648991449...|
|0.9979187729598386|0.002081227040161...|
|0.9979188325441605|0.002081167455839518|
|0.9979021097204755|0.002097890279524491|
| 0.997903704141627|0.002096295858372...|
|0.9977551595296489|0.002244840470350993|
|0.9977856537752533|0.002214346224746...|
| 0.997833713498837|0.002166286501163053|
|  0.99790306682705|0.002096933172949...|
|0.9978781800077882|0.002121819992211...|
|0.9976421530688276|0.002357846931172201|
|0.9979033017518185|0.002096698248181...|
+------------------+--------------

In [131]:
output3 = predictions.select('user_id', 'item_id', split2_udf('probability').alias('purchase')).sort(col("user_id").asc(), col("item_id").asc())

output3.show()

+-------+-------+--------------------+
|user_id|item_id|            purchase|
+-------+-------+--------------------+
|   1654|    326|0.002129589647197...|
|   1654|    546|0.002131562821532711|
|   1654|    578|0.002128008501577888|
|   1654|   1129| 0.00211705904692094|
|   1654|   1151|0.002128412907188269|
|   1654|   1222|0.002112812656592212|
|   1654|   1320|0.002113101592667...|
|   1654|   1599|0.002118676603218...|
|   1654|   2304|0.002178204159580919|
|   1654|   2327|0.002144677004588...|
|   1654|   2562|0.002129773887399...|
|   1654|   2736|0.002128469120898...|
|   1654|   2785|0.002129390677483448|
|   1654|   3099|0.002113431679722631|
|   1654|   3159|0.002114995337311...|
|   1654|   3408|0.002128377125996152|
|   1654|   3686|0.002134608744897...|
|   1654|   3692|0.002128329362527...|
|   1654|   3764|0.002159065111895...|
|   1654|   4255|0.002303861680495...|
+-------+-------+--------------------+
only showing top 20 rows



**Получаем предсказания для тестовой выборки**

In [132]:
test.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



In [133]:
test_temp = test.join(items, on="item_id", how="left")
test_temp.registerTempTable("test_temp")

In [134]:
test_temp = test_temp.join(item_popularity, on="item_id", how="left")
test_temp = test_temp.join(user_popularity, on="user_id", how="left")

In [135]:
test_temp.show(5)

+-------+-------+--------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+--------------------+---------+--------------------+--------------------+
|user_id|item_id|purchase|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|               title|  year|              genres|region_id|  mean_item_purchase|  mean_user_purchase|
+-------+-------+--------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+--------------------+---------+--------------------+--------------------+
| 867363| 100263|    null|      null|       1970-01-01T00:00:00Z|      2099-12-31T21:00:00Z|               null|              null|           1|           фортитьюд|2015.0|Ужасы,Детективы,Т...|     null|7.137758743754461E-4|3.892565200467107...|
| 867363| 100735

In [139]:
# обрабатываем genres
from pyspark.ml.feature import StringIndexer

test_temp = test_temp.na.fill({'genres':'NA'})
indexer = StringIndexer(inputCol="genres", outputCol="genre_id") 
test_temp = indexer.fit(test_temp).transform(test_temp) 
test_temp = test_temp.drop("genres").cache()
test_temp.show(2)

+-------+-------+--------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+---------+--------------------+--------------------+--------+
|user_id|item_id|purchase|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|               title|  year|region_id|  mean_item_purchase|  mean_user_purchase|genre_id|
+-------+-------+--------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+---------+--------------------+--------------------+--------+
| 867363| 100263|    null|      null|       1970-01-01T00:00:00Z|      2099-12-31T21:00:00Z|               null|              null|           1|           фортитьюд|2015.0|     null|7.137758743754461E-4|3.892565200467107...|   211.0|
| 867363| 100735|    null|      null|       1970-01-01T00:00:00Z

In [140]:
test_temp = test_temp.select(F.col("item_id").cast("int"),
                               F.col("mean_item_purchase").cast("float"),
                               F.col("user_id").cast("int"),
                               F.col("mean_user_purchase").cast("float"),
                               F.col("purchase").alias("label").cast("int"),
                               F.col("year").cast("int"),
                               F.col("genre_id").cast("int"))                              

In [141]:
test_temp.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- mean_item_purchase: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- mean_user_purchase: float (nullable = true)
 |-- label: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- genre_id: integer (nullable = true)



In [142]:
test_temp = test_temp.na.fill({'item_id':-1, 'user_id':-1, 
                                 'year':1970,'label':0,
                                 'mean_user_purchase':0, 'mean_item_purchase':0})
test_temp = test_temp.cache()

In [143]:
assemblerInputs = ['mean_item_purchase', 'mean_user_purchase', 'year', 'genre_id',]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [145]:
test_temp = assembler.transform(test_temp)

In [146]:
test_2 = test_temp.select("user_id","item_id","label", "features").cache()
test_2.show(2)

+-------+-------+-----+--------------------+
|user_id|item_id|label|            features|
+-------+-------+-----+--------------------+
| 867363| 100263|    0|[7.13775865733623...|
| 867363| 100735|    0|[7.51314801163971...|
+-------+-------+-----+--------------------+
only showing top 2 rows



In [147]:
predictions_test = lrModel.transform(test_2)

In [148]:
output = predictions_test.select('user_id', 'item_id', split2_udf('probability').alias('purchase')).sort(col("user_id").asc(), col("item_id").asc())

output.show()

+-------+-------+--------------------+
|user_id|item_id|            purchase|
+-------+-------+--------------------+
|   1654|    336|0.002119602484666343|
|   1654|    678|0.002113261943042748|
|   1654|    691|0.002115212340866...|
|   1654|    696|0.002144055122632...|
|   1654|    763|0.002128888764552467|
|   1654|    795|0.002253850565441...|
|   1654|    861|0.002128437574394...|
|   1654|   1137|0.002176917673465034|
|   1654|   1159|0.002143780491118...|
|   1654|   1428|0.002129223734127575|
|   1654|   1685|0.002150379840219...|
|   1654|   1686|0.002128689955180...|
|   1654|   1704|0.002159207567787...|
|   1654|   2093|0.002114690855248161|
|   1654|   2343|0.002128177669228...|
|   1654|   2451|0.002116602505044414|
|   1654|   2469|0.002338780263039...|
|   1654|   2603|0.002132505102742692|
|   1654|   2609|0.002120341450096642|
|   1654|   2621|0.002143659616926...|
+-------+-------+--------------------+
only showing top 20 rows



In [170]:
output.toPandas().to_csv('lab03.csv')

In [171]:
output.write.csv('output.csv')

In [160]:
preds = output.toPandas()
preds.head()

,user_id,item_id,purchase
0,1654,336,0.002120
1,1654,678,0.002113
2,1654,691,0.002115
3,1654,696,0.002144
4,1654,763,0.002129


In [168]:
preds = preds.to_dict()

AttributeError: 'dict' object has no attribute 'to_dict'

**Записываем предсказания в файл**

In [169]:
import json

with open('lab03.json', 'w', encoding='utf-8') as w:
    json.dump(preds, w, ensure_ascii=False, indent=4)

In [ ]:
sc.stop()